# IOTA microgrid tests

In [1]:
from iota import ProposedTransaction, Address, Tag, TryteString, Iota, Transaction
from iota.crypto.types import Seed
import json
import pandapower as pp
import numpy as np
from numpy.random import rand
import pandas as pd
import os
import pprint

### General params

In [2]:
num_household = 6
num_agents = 6
steps = 5

In [3]:
my_seed = Seed.random()
api = Iota('https://nodes.devnet.iota.org:443')
total_addresses = steps * (2 * num_agents + 1)
addresses = api.get_new_addresses(count=total_addresses)['addresses']

In [4]:
price_addresses_splits = []
money_addresses_splits = []
addresses1 = addresses[30:]
addresses2 = addresses[:30]
for agent in range(0, num_agents):
    price_addresses_splits.append(addresses1[agent*steps:agent*steps+steps])
    money_addresses_splits.append(addresses2[agent*steps:agent*steps+steps])
publish_addresses = addresses1[-steps:]

In [5]:
type(publish_addresses[0])

iota.types.Address

In [6]:
price_addresses_splits
money_addresses_splits
publish_addresses

[Address(b'ROFFFUMVVGJ9QRYQTTI9OGNEXCMQFQMIGVVESS9KIROOEKJVQQHHURMRBHTKQD9HNGHJSVXIDYE9HKJWX'),
 Address(b'NWZKTFZKPYFBEATBOSA9HTWCJXLSZVOGFJVMLYYXEBXEPBTDKDJWHYLUAF9NXUELQPC9YSRL99GC9OGVY'),
 Address(b'FAGPRXH9ZXFVMHPVLJUOGABJELHDGRYWOF9CXFFWWJMOAITWQBCTZTFEHNGOMNEJJEIBRMAKEIWDFJPOD'),
 Address(b'9ZDXGXEQROBLXBBRWFRXAIKIPUGJJJPUQLWQJBCLULIYUV9XNHDWWCFMD99FDPPYSBLDD9JPJEGLEDNXW'),
 Address(b'KBBICXM9CGMCHXZVCVBILTATUJXYLPACJLTA9SCHHXUEBSDHBJHMNPYDVODB9SNRXZNNHJEHITVDKLEFD')]

### Agent class

In [7]:
class Agent:
    def __init__(self, demand, supply, node, publish_address, price_address, money_address, price):
        self.demand = demand
        self.supply = supply
        self.node = node
        self.publish_address = publish_address
        self.price_address = price_address
        self.money_address = money_address
        self.consumption = demand-supply
        self.price = price
    
    def publish_info(self, step, data, address):
        tx = ProposedTransaction(
            address = Address(address),
            message = TryteString.from_unicode(json.dumps(data)),
            tag = Tag('INFO'),
            value = 0
        )

        tx = api.prepare_transfer(transfers=[tx])
        result = api.send_trytes(tx['trytes'], depth=3, min_weight_magnitude=9)
        return result
    
    def pay_power(self, step):
        unpack_data = self.check_address(step, 'price')
        print(unpack_data)
        for index, node in enumerate(unpack_data[0]['node']):
            data_to_send = {'payment': unpack_data[0]['price']*unpack_data[0]['power'][index]}
            address = self.address_dict[node]['money_address'][step]
            self.publish_info(step, data_to_send, address)
        
    def publish_energy_info(self, step):
        data = {'node': self.node,
                'demand': self.demand[step],
                'supply': self.supply[step],
                'consumption': self.consumption[step],
                'price': self.price[step]}
        address = self.publish_address[step]
        self.publish_info(step, data, address)
        return data
    
    def get_demand(self):
        return self.demand
    
    def get_supply(self):
        return self.supply
    
    def get_node(self):
        return self.node
    
    def get_price_address(self):
        return self.price_address
    
    def get_money_address(self):
        return self.money_address
    
    def get_consumption(self):
        return self.consumption
    
    def get_prices(self):
        return self.price
    
    def assign_address_dict(self, address_dict):
        self.address_dict = address_dict
    
    def check_address(self, step, address_type):
        
        if address_type == 'price':
            address = self.price_address[step]
        elif address_type == 'money':
            address = self.money_address[step]
        
        transactions = api.find_transactions(addresses=[address,])

        hashes = []
        for txhash in transactions['hashes']:
            hashes.append(txhash)

        trytes = api.get_trytes(hashes)['trytes']

        parts = []
        for trytestring in trytes:
            tx = Transaction.from_tryte_string(trytestring)
            parts.append((tx.current_index, tx.signature_message_fragment))

        parts.sort(key=lambda x: x[0])

        full_message = TryteString.from_unicode('')

        retrieved_data = []
        for index, part in parts:
            retrieved_data.append(json.loads(part.decode(errors='ignore')))
        return retrieved_data

### Loads, generations and prices

In [8]:
load_base = [5,6,8,7,4]
loads = []
for i in range(num_agents):
    rand_list = load_base + rand(5)
    loads.append(rand_list)
total_load = sum(loads)

In [10]:
gens = [i+4/4 for i in total_load]
gens = [gens, [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], gens, gens, gens]

In [9]:
prices = [[1.5, 1.5, 1.7, 1.5, 1.5],
          [0,0,0,0,0],
          [0,0,0,0,0],
          [1.2, 1.1, 1.2, 1.6, 1.6],
          [1.5, 1.5, 1.7, 1.5, 1.5],
          [1.5, 1.5, 1.7, 1.5, 1.5],
          [1.5, 1.4, 1.3, 1.2, 1.1]]

### Agent creation

In [10]:
def build_address_dictionary(agents, publish_addresses):
    agent_addresses = {}
    agent_addresses['publish_address'] = publish_addresses
    for index, agent in enumerate(agents):
        agent_addresses[index] = {'price_address': agent.get_price_address(),
                                  'money_address': agent.get_money_address()}
    return agent_addresses

In [11]:
agents = [Agent(loads[0], gens[0], 0, publish_addresses, price_addresses_splits[0], money_addresses_splits[0], prices[0]),
          Agent(loads[1], gens[1], 1, publish_addresses, price_addresses_splits[1], money_addresses_splits[1], prices[1]),
          Agent(loads[2], gens[2], 2, publish_addresses, price_addresses_splits[2], money_addresses_splits[2], prices[2]),
          Agent(loads[3], gens[3], 3, publish_addresses, price_addresses_splits[3], money_addresses_splits[3], prices[3]),
          Agent(loads[4], gens[4], 4, publish_addresses, price_addresses_splits[4], money_addresses_splits[4], prices[4]),
          Agent(loads[5], gens[5], 5, publish_addresses, price_addresses_splits[5], money_addresses_splits[5], prices[5])]

In [12]:
addresses_dict = build_address_dictionary(agents, publish_addresses)
addresses_dict

{'publish_address': [Address(b'FIIHLRPXJYHONNIDK9EEXOJBCZCDHPWFHRWMXKOPWEHUXSSYGFTOMLHRMPZY9Z9DJ99UVUIKKUZHFLUBX'),
  Address(b'GHEHLRMURNJKRGNRJMHTVTREJSHBZTXCPRVBGWPLDTJXTPUIEKQUZAZ9UBHGFVUTNZNSGOEENCJELAZGZ'),
  Address(b'REJCXVQHOYVWQJUDPEDFKWIHNYEIUHGPJOVILUXITLKUHCHOQVRGZARODMCBFVGKORHSIBXQGLLCDGL9X'),
  Address(b'PVORFPHCLBAGSIPQMTQBAAHPIR9YFMJQDTJIUDPERJQMF9CDMMSETHVYFZMZVFMVFSXSXCVCYKXYDMJEW'),
  Address(b'ZLZVHBOYTUXEVSKQSUXNHZKCZMEZRKKTIHAEJVKTHES9QTXVFBLDBJUBSIQCDVHFELZAXMXJHIBEHBOAW')],
 0: {'price_address': [Address(b'MHINXEFKQXASMVSYSMBCVDXFGTIOKXMJTYXTFFEGQZDZWXVALDCUJGIZLH9RXRPSUORMW9CZP99OWRTZW'),
   Address(b'JUBQLQHWTSVBK9UYUDUSWJIVBRSLCHVUABELTUSWWFRDI9HWDVJDCBWYNRKJJPYFFQQ9VURTMPGZVYL9B'),
   Address(b'LDDGORPQBHPJHKKZFFVUGTMPSFRDOJNUYCTLWMBTFRTXUZLQOQ9NUFAJBAIHRNQSYMTPERXPHSQBMVPCC'),
   Address(b'9ISHUXHLWKOPTKDJVXPUBUX9XOBA9HRIOXNBSC99NZDEKOYETTRUZWJMDOATYALNUXRIWPUWN99ODBEGX'),
   Address(b'TD9NZUKZOSVWTSZBCECDORCKUKDAOCOICMAMMLYKQQLHBRIB9YJTHHADVJKWV9QQNDZBLF

In [13]:
for agent in agents:
    agent.assign_address_dict(addresses_dict)

In [14]:
print(agents[1].get_price_address())

[Address(b'IQJ9LOOD9DPWOEFGRBYBEFDQDPVGFLKUUJGVVUEVUPWULXQWOJRAARXZXLQOITEKIHSRHWOULAPGEQQ99'), Address(b'URHIEOMWDTF9J9XLUSBMJUKVDPCQICLLMWAALNKKRLFRBFRBET9UCLUIAO9MBWXU9BXDUIDSDRRLGISXD'), Address(b'GTMBFCACPXLYWOYJM9HCOOOWXCOQIFQWFRFLJCHGWTWMHVQDMNPPWBZGRHVXKUIFVZPRGICKTIBB9NHID'), Address(b'PYDLIBFJUSRPOVDHERXNUWC9XWJXSCPUR9OVTGEOEBQQDRVNTBOAHAJWGLPLDYWWAA9WCRVLOKLYFVAGB'), Address(b'VTOVJKHV9DVQPJQYFLOPAOYRIADNDBRXLVWZQKPOZZOMKMEXWDSHGBBQHLDUWLVIQCLDEGFF9LW9WRLVY')]


Each agent publishes the necessary info to the main address

In [15]:
for index, agent in enumerate(agents):
    agent.publish_energy_info(0)

Retrieve the published information

In [16]:
def check_published_data(address):
    transactions = api.find_transactions(addresses=[address,])

    hashes = []
    for txhash in transactions['hashes']:
        hashes.append(txhash)
        
    trytes = api.get_trytes(hashes)['trytes']

    parts = []
    for trytestring in trytes:
        tx = Transaction.from_tryte_string(trytestring)
        parts.append((tx.current_index, tx.signature_message_fragment))

    parts.sort(key=lambda x: x[0])

    full_message = TryteString.from_unicode('')

    node_data = []
    for index, part in parts:
#         pprint.pprint(json.loads(part.decode(errors='ignore')))8.299+8.299+8.299+8.154
        node_data.append(json.loads(part.decode(errors='ignore')))

    return node_data

In [17]:
node_data = check_published_data(publish_addresses[0])
node_data

[{'node': 5,
  'demand': 5.431409908230468,
  'supply': 34.459134033585386,
  'consumption': -29.02772412535492,
  'price': 1.5},
 {'node': 0,
  'demand': 5.585266518288668,
  'supply': 34.459134033585386,
  'consumption': -28.873867515296716,
  'price': 1.5},
 {'node': 1,
  'demand': 5.336467588910152,
  'supply': 0,
  'consumption': 5.336467588910152,
  'price': 0},
 {'node': 2,
  'demand': 5.451319419347989,
  'supply': 0,
  'consumption': 5.451319419347989,
  'price': 0},
 {'node': 3,
  'demand': 5.661050700564067,
  'supply': 34.459134033585386,
  'consumption': -28.798083333021317,
  'price': 1.2},
 {'node': 4,
  'demand': 5.993619898244042,
  'supply': 34.459134033585386,
  'consumption': -28.465514135341344,
  'price': 1.5}]

In [18]:
def single_sided_auction(node_data):
    node_vec = []
    supply_vec = []
    supply_price = []
    demand = []
    ED = 0
    for node in node_data:
        node_vec.append(node['node'])
        supply_vec.append(node['supply'])
        supply_price.append(node['price'])
        demand.append(node['demand'])
        ED += node['demand']
    supply_df = pd.DataFrame(data = {'Agents': node_vec, 'Supply': supply_vec, 'Price': supply_price, 'Demand': demand})
    supply_df = supply_df.sort_values(by='Price')
    cum_supply = np.cumsum(supply_df['Supply'])
    supply_df['Cumsum'] = cum_supply
    for index, value in enumerate(supply_df.Cumsum):
        if value >= ED:
            break
    return supply_df, ED, supply_df.iloc[index]['Price']

In [19]:
supply_df, ED, price = single_sided_auction(node_data)
supply_df.set_index('Agents', inplace=True)
supply_df

,Supply,Price,Demand,Cumsum
Agents,,,,
1,0.000000,0.0,5.336468,0.000000
2,0.000000,0.0,5.451319,0.000000
3,34.459134,1.2,5.661051,34.459134
5,34.459134,1.5,5.431410,68.918268
0,34.459134,1.5,5.585267,103.377402
4,34.459134,1.5,5.993620,137.836536



### Microgrid nodes

In [20]:
net = pp.create_empty_network()
min_pu = 0.95
max_pu = 1.05

bus0 = pp.create_bus(net, vn_kv=110, min_vm_pu=min_pu, max_vm_pu=max_pu)
bus1 = pp.create_bus(net, vn_kv=110, min_vm_pu=min_pu, max_vm_pu=max_pu)
bus2 = pp.create_bus(net, vn_kv=110, min_vm_pu=min_pu, max_vm_pu=max_pu)
bus3 = pp.create_bus(net, vn_kv=110, min_vm_pu=min_pu, max_vm_pu=max_pu)
bus4 = pp.create_bus(net, vn_kv=110, min_vm_pu=min_pu, max_vm_pu=max_pu)
bus5 = pp.create_bus(net, vn_kv=110, min_vm_pu=min_pu, max_vm_pu=max_pu)

line1 = pp.create_line(net, bus0, bus1, length_km=1, std_type='149-AL1/24-ST1A 110.0')
line2 = pp.create_line(net, bus1, bus2, length_km=1, std_type='149-AL1/24-ST1A 110.0')
line3 = pp.create_line(net, bus2, bus3, length_km=1, std_type='149-AL1/24-ST1A 110.0')
line4 = pp.create_line(net, bus3, bus4, length_km=1, std_type='149-AL1/24-ST1A 110.0')
line5 = pp.create_line(net, bus4, bus5, length_km=1, std_type='149-AL1/24-ST1A 110.0')
line6 = pp.create_line(net, bus5, bus0, length_km=1, std_type='149-AL1/24-ST1A 110.0')

pp.create_load(net, bus0, p_mw=supply_df['Demand'][0])
pp.create_load(net, bus1, p_mw=supply_df['Demand'][1])
pp.create_load(net, bus2, p_mw=supply_df['Demand'][2])
pp.create_load(net, bus3, p_mw=supply_df['Demand'][3])
pp.create_load(net, bus4, p_mw=supply_df['Demand'][4])
pp.create_load(net, bus5, p_mw=supply_df['Demand'][5])

gen0 = pp.create_gen(net, bus0, p_mw=30, min_p_mw=0, max_p_mw=32, controllable=True, slack=True)
gen3 = pp.create_gen(net, bus3, p_mw=30, min_p_mw=0, max_p_mw=32, controllable=True, slack=True)
gen4 = pp.create_gen(net, bus4, p_mw=30, min_p_mw=0, max_p_mw=32, controllable=True, slack=True)
gen5 = pp.create_gen(net, bus5, p_mw=30, min_p_mw=0, max_p_mw=32, controllable=True, slack=True)

In [21]:
pp.runpp(net)
pf_result = net.res_gen
pf_result

,p_mw,q_mvar,va_degree,vm_pu
0,10.960482,-0.065536,0.0,1.0
1,11.074555,-0.065527,0.0,1.0
2,5.993620,-0.033262,0.0,1.0
3,5.431410,-0.033262,0.0,1.0


In [22]:
supply_df

,Supply,Price,Demand,Cumsum
Agents,,,,
1,0.000000,0.0,5.336468,0.000000
2,0.000000,0.0,5.451319,0.000000
3,34.459134,1.2,5.661051,34.459134
5,34.459134,1.5,5.431410,68.918268
0,34.459134,1.5,5.585267,103.377402
4,34.459134,1.5,5.993620,137.836536


In [23]:
gen_nodes = [0, 3, 4, 5]
gen_values = pf_result['p_mw']
gen_dict = dict(zip(gen_nodes, gen_values))

In [24]:
gen_dict

{0: 10.96048243976144,
 3: 11.074554951772061,
 4: 5.993619898244042,
 5: 5.431409908230471}

In [25]:
total_demand = sum(supply_df['Demand'])
total_supply = sum(gen_dict.values())
losses = total_supply - total_demand

In [26]:
losses

0.0009331644226264757

In [27]:
power_per_agent = []
for agent in agents:
    print(agent.get_node())
    demand = agent.demand[0]
    try:
        supply = gen_dict[agent.get_node()]
    except:
        supply = agent.get_supply()[0]
    print('Supply {} Demand {}'.format(supply, demand))
    to_app = supply - demand
    if abs(to_app) < 0.00001:
        to_app = 0
    power_per_agent.append(to_app)

0
Supply 10.96048243976144 Demand 5.585266518288668
1
Supply 0 Demand 5.336467588910152
2
Supply 0 Demand 5.451319419347989
3
Supply 11.074554951772061 Demand 5.661050700564067
4
Supply 5.993619898244042 Demand 5.993619898244042
5
Supply 5.431409908230471 Demand 5.431409908230468


In [28]:
power_per_agent

[5.375215921472773,
 -5.336467588910152,
 -5.451319419347989,
 5.4135042512079945,
 0,
 0]

In [29]:
pay_for_power = []
for power in power_per_agent:
    pay_for_power.append(power*price)

In [30]:
pay_for_power

[6.450259105767327,
 -6.403761106692182,
 -6.541583303217586,
 6.496205101449593,
 0.0,
 0.0]

In [31]:
to_earn = {}
to_pay = {}
for index, payment in enumerate(power_per_agent):
    if payment < 0:
        to_pay[index] = abs(payment)
    else:
        to_earn[index] = payment

In [32]:
to_earn

{0: 5.375215921472773, 3: 5.4135042512079945, 4: 0, 5: 0}

In [33]:
to_pay

{1: 5.336467588910152, 2: 5.451319419347989}

In [34]:
payment_info = {}
for payer, balance in to_pay.items():
    payment_info[payer] = {'node': [], 'total': []}
    for earner, gain in to_earn.items():
        if gain > 0:
            if balance >= gain:
                payment_info[payer]['node'].append(earner)
                payment_info[payer]['total'].append(gain)
                to_pay[payer] = to_pay[payer] - gain
                to_earn[earner] = to_earn[earner] - gain
            elif balance < gain:
                payment_info[payer]['node'].append(earner)
                payment_info[payer]['total'].append(balance)
                to_pay[payer] = to_pay[payer] - balance
                to_earn[earner] = to_earn[earner] - balance
                break

In [35]:
payment_info

{1: {'node': [0], 'total': [5.336467588910152]},
 2: {'node': [0, 3], 'total': [0.038748332562621, 5.4135042512079945]}}

In [36]:
gather_data = {}
for index, agent in enumerate(agents):
    info_dict = {}
    info_dict['price'] = price
    try:
        info_dict['node'] = payment_info[index]['node']
        info_dict['power'] = payment_info[index]['total']
    except:
        pass
    gather_data[index] = info_dict

In [37]:
gather_data

{0: {'price': 1.2},
 1: {'price': 1.2, 'node': [0], 'power': [5.336467588910152]},
 2: {'price': 1.2,
  'node': [0, 3],
  'power': [0.038748332562621, 5.4135042512079945]},
 3: {'price': 1.2},
 4: {'price': 1.2},
 5: {'price': 1.2}}

In [38]:
step = 0
for index, agent in enumerate(agents):
    address = addresses_dict[index]['price_address'][step]
    data = gather_data[index]
    tx = ProposedTransaction(
        address=Address(address),
        message=TryteString.from_unicode(json.dumps(data)),
        tag=Tag('PRICE'),
        value=0)
        
    tx = api.prepare_transfer(transfers=[tx])

    result = api.send_trytes(tx['trytes'], depth=3, min_weight_magnitude=9)
    
    print(address)

MHINXEFKQXASMVSYSMBCVDXFGTIOKXMJTYXTFFEGQZDZWXVALDCUJGIZLH9RXRPSUORMW9CZP99OWRTZW
IQJ9LOOD9DPWOEFGRBYBEFDQDPVGFLKUUJGVVUEVUPWULXQWOJRAARXZXLQOITEKIHSRHWOULAPGEQQ99
NUGF9CWMNGZPTYQNAK9NPEDTOCTQUYVZMOPAMREEKLRKM9WWSQTEAEAIFI9IVNBSUYGFZHSCWXZXJAESX
JCDVLQSRZNJYQQ99YR9VEMJNMYBIVWTPJTVUB9PHQZHGLQIMCZYPNMFXUWBACHNOLS9TEITZSZRP9TVPD
JSQQFRMNFX9VCIYMWUFUGZNKBWVMROMTZXPSRKIWMWRBQIHAVRKHRZMO9TEEWWQEENAQYQQDLTFSGGJKC
XVJWOGXPS9RMVO9HXDKLIHMHAY9WTVBXABMGDIPDRXACWFNTJLNIMBWHGTUPEOHEHCGGRXNZ9TDCK9CHC


In [39]:
step = 0
for agent in agents:
    print(agent.check_address(step, 'price'))

[{'price': 1.2}]
[{'price': 1.2, 'node': [0], 'power': [5.336467588910152]}]
[{'price': 1.2, 'node': [0, 3], 'power': [0.038748332562621, 5.4135042512079945]}]
[{'price': 1.2}]
[{'price': 1.2}]
[{'price': 1.2}]


In [40]:
step = 0
agents[1].pay_power(step)
agents[2].pay_power(step)

[{'price': 1.2, 'node': [0], 'power': [5.336467588910152]}]
[{'price': 1.2, 'node': [0, 3], 'power': [0.038748332562621, 5.4135042512079945]}]


In [41]:
step = 0
print(agents[0].check_address(step, 'money'))
print(agents[3].check_address(step, 'money'))

[{'payment': 6.403761106692182}, {'payment': 0.0464979990751452}]
[{'payment': 6.496205101449593}]


In [ ]:
def payment_setup():
    power_per_agent = []
    for agent in agents:
        agent.get_node()
        demand = agent.demand[0]
        try:
            supply = gen_dict[agent.get_node()]
        except:
            supply = agent.get_supply()[0]
        to_app = supply - demand
        if abs(to_app) < 0.00001:
            to_app = 0
        power_per_agent.append(to_app)
    to_earn = {}
    to_pay = {}
    for index, payment in enumerate(power_per_agent):
        if payment < 0:
            to_pay[index] = abs(payment)
        else:
            to_earn[index] = payment
    
    payment_info = {}
    for payer, balance in to_pay.items():
        payment_info[payer] = {'node': [], 'total': []}
        for earner, gain in to_earn.items():
            if gain > 0:
                if balance >= gain:
                    payment_info[payer]['node'].append(earner)
                    payment_info[payer]['total'].append(gain)
                    to_pay[payer] = to_pay[payer] - gain
                    to_earn[earner] = to_earn[earner] - gain
                elif balance < gain:
                    payment_info[payer]['node'].append(earner)
                    payment_info[payer]['total'].append(balance)
                    to_pay[payer] = to_pay[payer] - balance
                    to_earn[earner] = to_earn[earner] - balance
                    break
    gather_data = {}
    for index, agent in enumerate(agents):
        info_dict = {}
        info_dict['price'] = price
        try:
            info_dict['node'] = payment_info[index]['node']
            info_dict['power'] = payment_info[index]['total']
        except:
            pass
        gather_data[index] = info_dict